# Exercise 1: Introduction to large rotations in MeshPy

The beam finite element input generator **MeshPy** comes with a framework for handling of large rotations.

First, we need to import the relevant python packages and objects:

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from beamme.core.rotation import Rotation

The main workflow to handle large rotations is with the `Rotation` object.
This object represents an element of the $SO3$ group.
An identity rotation can be created with:

In [5]:
rotation_identity = Rotation()
print(rotation_identity)

Rotation:
    q0: 1.0
    q: [0. 0. 0.]


We can see that the internal representation of this rotation object is done with quaternions.
This does not hinder us from using any other common type of rotational parametrization.
We can also create the object from a rotation (pseudo-)vector:

In [6]:
rotation_vector_1 = [0.5 * np.pi, 0, 0]
rotation_1 = Rotation.from_rotation_vector(rotation_vector_1)
print(rotation_1)

Rotation:
    q0: 0.7071067811865476
    q: [0.70710678 0.         0.        ]


We can now get the rotation matrix for this rotation

In [7]:
print(rotation_1.get_rotation_matrix())

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.22044605e-16 -1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  2.22044605e-16]]


Let's define a second rotation

In [8]:
rotation_2 = Rotation.from_rotation_vector([0, 0.5 * np.pi, 0.5])

We now have two rotations $\Lambda_1$ and $\lambda_2$, represented by the variables `rotation_1` and `rotation_2`.
The compound rotation $\Lambda_3=\Lambda_2 \Lambda_1$ can be computed with

In [6]:
rotation_3 = rotation_2 * rotation_1
print(rotation_3)

Rotation:
    q0: 0.48021354734011684
    q: [ 0.48021355  0.65201147 -0.33715121]


We can have a look at the rotation vector and matrix representations

In [7]:
print(f"rotation_3 (rotation vector) = {rotation_3.get_rotation_vector()}")
print(f"rotation_3 (matrix) =\n{rotation_3.get_rotation_matrix()}")

rotation_3 (rotation vector) = [ 1.17147273  1.59057083 -0.82247461]
rotation_3 (matrix) =
[[-0.0775798   0.95001864  0.30240033]
 [ 0.30240033  0.31144802 -0.90086302]
 [-0.95001864  0.02155719 -0.31144802]]


We could have also computed $\Lambda_3$ from the rotation matrices of $\Lambda_1$ and $\Lambda_2$

In [8]:
rotation_1_rotation_matrix = rotation_1.get_rotation_matrix()
rotation_2_rotation_matrix = rotation_2.get_rotation_matrix()
rotation_3_rotation_matrix = np.dot(
    rotation_2_rotation_matrix, rotation_1_rotation_matrix
)
rotation_3_from_rotation_matrix = Rotation.from_rotation_matrix(
    rotation_3_rotation_matrix
)
if rotation_3 == rotation_3_from_rotation_matrix:
    print("Rotations are equal")
else:
    print("Rotations are not equal")

Rotations are equal


A rotation object can be _inverted_

In [9]:
rotation_2_inverse = rotation_2.inv()

With this we can compute $\Lambda_1=\Lambda_2^{-1}\Lambda_3$ which is equal to the original definition of $\Lambda_1$

In [10]:
rotation_1_alternative = rotation_2_inverse * rotation_3
if rotation_1_alternative == rotation_1:
    print("Rotations are equal")
else:
    print("Rotations are not equal")

Rotations are equal


We can also easily compute the rotation of a vector $a'=\Lambda_3 a$:

In [11]:
a = [1, 2, 3]
a_prime = rotation_3 * a
print(a_prime)

[ 2.72965846 -1.77729268 -1.84124833]


We get the same result if we use the rotation matrix directly

In [12]:
a_prime_alternative = np.dot(rotation_3.get_rotation_matrix(), a)
print(a_prime_alternative)

[ 2.72965846 -1.77729268 -1.84124833]
